In [ ]:
using Statistics
using NiLang
using BenchmarkTools


In [ ]:
#提取日线沪深300指数收盘价
a= readlines("F:\\我的坚果云\\julia\\策略微分\\data.txt");
b=parse.(Float64,a);
b2=copy(b);


In [18]:
#EMA权重
bili=0.0952;
n=length(b);
ma=zeros(n,1);
ma[1]=b[1];
pos=zeros(n,1);
#单位净值
jinzhi=ones(n,1);
spread=zeros(n,1);
ret=b[2:end]./b[1:end-1].-1;
for i =2:n
    if pos[i-1]>0
        #如果信号为正,做多
        jinzhi[i]=jinzhi[i-1]*(b[i]/b[i-1]);
    elseif pos[i-1]<0
        #如果信号为负,做空
        jinzhi[i]=jinzhi[i-1]*(2-b[i]/b[i-1]);
    else
        #如果信号为0,保持不变
        jinzhi[i]=jinzhi[i-1];
    end
    #计算EMA
    ma[i]=b[i]*bili+(1-bili)*ma[i-1];
    spread[i]=b[i]-ma[i];
    pos[i] = -tanh(ma[i]-b[i]);

end


In [31]:
#计算t期的spread和t+1期股指return的皮尔逊相关性
cor(spread[1:end-1],ret)

0.034088594834217625

In [24]:
@i function jin(out::T,b::AbstractVector{T},ma::AbstractVector{T},jinzhi::AbstractVector{T},spread::AbstractVector{T},bili::T) where T
    @routine begin
        for i =2:length(b)

            if (spread[i-1]!=0, ~)
                @routine begin
                    @zeros T anc1 
                    anc1 += b[i] / b[i-1]
                    anc1 -= 1.0
                end

                jinzhi[i] += anc1
                ~@routine
            end

            ma[i] += b[i] * bili
            bili -= 1.0
            ma[i] -= bili * ma[i-1]
            bili += 1.0


            # the relaxed version
            ma[i] -= b[i]
            spread[i] -= ma[i]
            ma[i] += b[i]
        end
        #计算t期的spread和t+1期股指return的皮尔逊相关性
        @zeros T var1 varsum1 mean1 sum1 std1
        NiLang.i_var_mean_sum(var1, varsum1, mean1, sum1, jinzhi[2:end])
        std1 += sqrt(var1)
        @zeros T var2 varsum2 mean2 sum2 std2
        NiLang.i_var_mean_sum(var2, varsum2, mean2, sum2, spread[1:end-1])
        std2 += sqrt(var2)
        @zeros T anc3 anc4 anc5 anc6 anc7
        #计算协方差矩阵
        for i=2:length(b)
            jinzhi[i] -= mean1
            anc3 +=jinzhi[i]
            jinzhi[i] +=mean1
            spread[i-1] -=mean2
            anc4 +=spread[i-1]
            spread[i-1] +=mean2
            anc5 += anc3*anc4     
        end
        anc6+=std1*std2
        anc7+=anc6*(length(jinzhi)-2)
    end
    #计算相关性
    out += anc5/anc7 
    ~@routine
end
    

In [25]:
bili=0.0952;
n=length(b);
ma=zeros(n);
ma[1]=b[1];
jinzhi=zeros(n);
spread=zeros(n);
out=0.0;
# check the reversibility
check_inv(jin, (out,b,ma,jinzhi,spread,bili))


true

In [26]:
# check the gradients with numeric gradients
NiLang.AD.check_grad(jin, (out,b,ma,jinzhi,spread,bili); iloss=1)

InvertibilityError: InvertibilityError("x (=1.573760499695709e-8) ≂̸ val (=0.0)")

In [ ]:
out,b,ma,jinzhi,spread,bili=jin(out,b,ma,jinzhi,spread,bili)


In [ ]:
out,b,ma,jinzhi,spread,bili=(~jin)(out,b,ma,jinzhi,spread,bili)

In [ ]:
gout,gb,gma,gjinzhi,gspread,gbili= NiLang.AD.gradient(Val(1), jin, (out,b,ma,jinzhi,spread,bili))